# Configuracion del entorno

In [1]:
import sys
print(sys.executable)

/mnt/homeGPU/haoweihu/env/bin/python


In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="<0>"


In [3]:
# The first thing we should do is to apply hardware acceleration via GPU. So, we have to go to Editar - Configuracion del Cuaderno - Acelerador por Hardware GPU

import fastbook
fastbook.setup_book()
import fastai
import fastcore
import cv2

print(f'fastcore version {fastcore.__version__} installed')
print(f'fastai version {fastai.__version__} installed')
from nbdev.showdoc import *
from fastai.vision.all import *

fastcore version 1.7.27 installed
fastai version 2.7.19 installed


In [6]:
class LeNet(nn.Module):
  def __init__(self, num_classes=10):
    super(LeNet, self).__init__()
    self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
    self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
    self.fc1 = nn.Linear(16 * 4 * 4, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, num_classes)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)
    x = x.view(x.size(0), -1)  # Aplanar la salida
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [16]:
def umbralizacion_tri(img: PILImage):
    return img.quantize(colors=3)

def umbralizacion_bi(img: PILImage):
    return img.quantize(colors=2)

import cv2 
def otsu_threshold_transform(img: PILImage):
    #img_gray = img.convert("L")
    img_array = np.array(img)
    _, thresholded = cv2.threshold(img_array, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return PILImage.create(thresholded)

In [4]:
from fastai.vision.all import *
from pathlib import Path
import time

def quantize3(train_path, name):

    # Crear el DataBlock para entrenamiento y validación
    dblock = DataBlock(
        blocks=(ImageBlock(cls=PILImageBW), CategoryBlock),
        get_items=get_image_files,
        splitter=GrandparentSplitter(train_name='training', valid_name='valid'),
        get_y=parent_label,
        item_tfms=[umbralizacion_tri]
    )

    dls = dblock.dataloaders(train_path, bs=64)

    # Obtener el número total de imágenes
    num_train = len(dls.train_ds)
    num_valid = len(dls.valid_ds)

    print(f"Número total de imágenes en entrenamiento: {num_train}")
    print(f"Número total de imágenes en validación: {num_valid}")

    # Obtener la distribución de clases
    class_counts = dls.train_ds.vocab  # Obtiene las clases
    print("\nClases:", class_counts)

    # Inicializar el modelo LeNet con 10 clases
    model = LeNet(num_classes=10)
    
    # Crear el objeto Learner
    learn = Learner(
        dls, 
        model, 
        loss_func=CrossEntropyLossFlat(), 
        metrics=[accuracy, Recall(average='macro'), F1Score(average='macro')], 
        cbs=[EarlyStoppingCallback(monitor='valid_loss', patience=5)]
    )

    # Entrenamiento del modelo con fit_one_cycle
    start_time = time.time()
    learn.fit_one_cycle(30)
    end_time = time.time()
    print(f"\nTiempo de entrenamiento: {end_time - start_time:.2f} segundos")

    # Validación y reporte de métricas
    loss, acc, recall, f1 = learn.validate()
    print(f'Loss: {loss:.4f}')
    print(f'Accuracy: {acc:.4f}')
    print(f"Recall: {recall:.4f}")
    print(f'F1-Score: {f1:.4f}')

    # Exportar el modelo entrenado
    learn.export('/mnt/homeGPU/haoweihu/code/models/'+name+'.pkl')

    # Crear el DataBlock para los datos de prueba
    test_block = DataBlock(
        blocks=(ImageBlock(cls=PILImageBW), CategoryBlock),
        get_items=get_image_files,
        get_y=parent_label,
        splitter=IndexSplitter([]),
        item_tfms=[umbralizacion_tri]
    )

    test_dls = test_block.dataloaders("/mnt/homeGPU/haoweihu/code/dataset/original/mnist_png/testing")

    # Cargar el modelo previamente exportado y remover el callback de EarlyStopping
    learn = load_learner('/mnt/homeGPU/haoweihu/code/models/'+name+'.pkl')
    learn.remove_cb(EarlyStoppingCallback)

    # Validar con el conjunto de test
    loss, acc, recall, f1 = learn.validate(dl=test_dls.train)
    print(f"Loss: {loss}")
    print(f"Accuracy: {acc}")
    print(f"Recall (macro): {recall}")
    print(f"F1 Score (macro): {f1}")
    


In [7]:
def quantize2(train_path, name):

    # Crear el DataBlock para entrenamiento y validación
    dblock = DataBlock(
        blocks=(ImageBlock(cls=PILImageBW), CategoryBlock),
        get_items=get_image_files,
        splitter=GrandparentSplitter(train_name='training', valid_name='valid'),
        get_y=parent_label,
        item_tfms=[umbralizacion_bi]
    )

    dls = dblock.dataloaders(train_path, bs=64)

    # Obtener el número total de imágenes
    num_train = len(dls.train_ds)
    num_valid = len(dls.valid_ds)

    print(f"Número total de imágenes en entrenamiento: {num_train}")
    print(f"Número total de imágenes en validación: {num_valid}")

    # Obtener la distribución de clases
    class_counts = dls.train_ds.vocab  # Obtiene las clases
    print("\nClases:", class_counts)

    # Inicializar el modelo LeNet con 10 clases
    model = LeNet(num_classes=10)
    
    # Crear el objeto Learner
    learn = Learner(
        dls, 
        model, 
        loss_func=CrossEntropyLossFlat(), 
        metrics=[accuracy, Recall(average='macro'), F1Score(average='macro')], 
        cbs=[EarlyStoppingCallback(monitor='valid_loss', patience=5)]
    )

    # Entrenamiento del modelo con fit_one_cycle
    start_time = time.time()
    learn.fit_one_cycle(30)
    end_time = time.time()
    print(f"\nTiempo de entrenamiento: {end_time - start_time:.2f} segundos")

    # Validación y reporte de métricas
    loss, acc, recall, f1 = learn.validate()
    print(f'Loss: {loss:.4f}')
    print(f'Accuracy: {acc:.4f}')
    print(f"Recall: {recall:.4f}")
    print(f'F1-Score: {f1:.4f}')

    # Exportar el modelo entrenado
    learn.export('/mnt/homeGPU/haoweihu/code/models/'+name+'.pkl')

    # Crear el DataBlock para los datos de prueba
    test_block = DataBlock(
        blocks=(ImageBlock(cls=PILImageBW), CategoryBlock),
        get_items=get_image_files,
        get_y=parent_label,
        splitter=IndexSplitter([]),
        item_tfms=[umbralizacion_bi]
    )

    test_dls = test_block.dataloaders("/mnt/homeGPU/haoweihu/code/dataset/original/mnist_png/testing")

    # Cargar el modelo previamente exportado y remover el callback de EarlyStopping
    learn = load_learner('/mnt/homeGPU/haoweihu/code/models/'+name+'.pkl')
    learn.remove_cb(EarlyStoppingCallback)

    # Validar con el conjunto de test
    loss, acc, recall, f1 = learn.validate(dl=test_dls.train)
    print(f"Loss: {loss}")
    print(f"Accuracy: {acc}")
    print(f"Recall (macro): {recall}")
    print(f"F1 Score (macro): {f1}")

In [8]:
def otsu(train_path, name):

    # Crear el DataBlock para entrenamiento y validación
    dblock = DataBlock(
        blocks=(ImageBlock(cls=PILImageBW), CategoryBlock),
        get_items=get_image_files,
        splitter=GrandparentSplitter(train_name='training', valid_name='valid'),
        get_y=parent_label,
        item_tfms=[otsu_threshold_transform]
    )

    dls = dblock.dataloaders(train_path, bs=64)

    # Obtener el número total de imágenes
    num_train = len(dls.train_ds)
    num_valid = len(dls.valid_ds)

    print(f"Número total de imágenes en entrenamiento: {num_train}")
    print(f"Número total de imágenes en validación: {num_valid}")

    # Obtener la distribución de clases
    class_counts = dls.train_ds.vocab  # Obtiene las clases
    print("\nClases:", class_counts)

    # Inicializar el modelo LeNet con 10 clases
    model = LeNet(num_classes=10)
    
    # Crear el objeto Learner
    learn = Learner(
        dls, 
        model, 
        loss_func=CrossEntropyLossFlat(), 
        metrics=[accuracy, Recall(average='macro'), F1Score(average='macro')], 
        cbs=[EarlyStoppingCallback(monitor='valid_loss', patience=5)]
    )

    # Entrenamiento del modelo con fit_one_cycle
    start_time = time.time()
    learn.fit_one_cycle(30)
    end_time = time.time()
    print(f"\nTiempo de entrenamiento: {end_time - start_time:.2f} segundos")

    # Validación y reporte de métricas
    loss, acc, recall, f1 = learn.validate()
    print(f'Loss: {loss:.4f}')
    print(f'Accuracy: {acc:.4f}')
    print(f"Recall: {recall:.4f}")
    print(f'F1-Score: {f1:.4f}')

    # Exportar el modelo entrenado
    learn.export('/mnt/homeGPU/haoweihu/code/models/'+name+'.pkl')

    # Crear el DataBlock para los datos de prueba
    test_block = DataBlock(
        blocks=(ImageBlock(cls=PILImageBW), CategoryBlock),
        get_items=get_image_files,
        get_y=parent_label,
        splitter=IndexSplitter([]),
        item_tfms=[otsu_threshold_transform]
    )

    test_dls = test_block.dataloaders("/mnt/homeGPU/haoweihu/code/dataset/original/mnist_png/testing")

    # Cargar el modelo previamente exportado y remover el callback de EarlyStopping
    learn = load_learner('/mnt/homeGPU/haoweihu/code/models/'+name+'.pkl')
    learn.remove_cb(EarlyStoppingCallback)

    # Validar con el conjunto de test
    loss, acc, recall, f1 = learn.validate(dl=test_dls.train)
    print(f"Loss: {loss}")
    print(f"Accuracy: {acc}")
    print(f"Recall (macro): {recall}")
    print(f"F1 Score (macro): {f1}")

In [4]:
#if __name__ == '__main__':
    #quantize3("/mnt/homeGPU/haoweihu/code/dataset/distilled_001/mnist/example1", "mnist_dist_quant3_ex1")

from fastai.vision.all import *
from pathlib import Path
import time


# Crear el DataBlock para entrenamiento y validación
dblock = DataBlock(
    blocks=(ImageBlock(cls=PILImageBW), CategoryBlock),
    get_items=get_image_files,
    splitter=GrandparentSplitter(train_name='training', valid_name='valid'),
    get_y=parent_label,
    item_tfms=[umbralizacion_tri]
)

dls = dblock.dataloaders("/mnt/homeGPU/haoweihu/code/dataset/distilled_001/mnist/example1", bs=64)

    # Obtener el número total de imágenes
num_train = len(dls.train_ds)
num_valid = len(dls.valid_ds)

print(f"Número total de imágenes en entrenamiento: {num_train}")
print(f"Número total de imágenes en validación: {num_valid}")

# Obtener la distribución de clases
class_counts = dls.train_ds.vocab  # Obtiene las clases
print("\nClases:", class_counts)

# Inicializar el modelo LeNet con 10 clases
model = LeNet(num_classes=10)
    
# Crear el objeto Learner
learn = Learner(
    dls, 
    model, 
    loss_func=CrossEntropyLossFlat(), 
    metrics=[accuracy, Recall(average='macro'), F1Score(average='macro')], 
    cbs=[EarlyStoppingCallback(monitor='valid_loss', patience=5)]
)

# Entrenamiento del modelo con fit_one_cycle
start_time = time.time()
learn.fit_one_cycle(30)
end_time = time.time()
print(f"\nTiempo de entrenamiento: {end_time - start_time:.2f} segundos")

# Validación y reporte de métricas
loss, acc, recall, f1 = learn.validate()
print(f'Loss: {loss:.4f}')
print(f'Accuracy: {acc:.4f}')
print(f"Recall: {recall:.4f}")
print(f'F1-Score: {f1:.4f}')

# Exportar el modelo entrenado
learn.export('/mnt/homeGPU/haoweihu/code/models/mnist_dist_quant3_ex1.pkl')

    # Crear el DataBlock para los datos de prueba
test_block = DataBlock(
    blocks=(ImageBlock(cls=PILImageBW), CategoryBlock),
    get_items=get_image_files,
    get_y=parent_label,
    splitter=IndexSplitter([]),
    item_tfms=[umbralizacion_tri]
)

test_dls = test_block.dataloaders("/mnt/homeGPU/haoweihu/code/dataset/original/mnist_png/testing")

# Cargar el modelo previamente exportado y remover el callback de EarlyStopping
learn = load_learner('/mnt/homeGPU/haoweihu/code/models/mnist_dist_quant3_ex1.pkl')
learn.remove_cb(EarlyStoppingCallback)

# Validar con el conjunto de test
loss, acc, recall, f1 = learn.validate(dl=test_dls.train)
print(f"Loss: {loss}")
print(f"Accuracy: {acc}")
print(f"Recall (macro): {recall}")
print(f"F1 Score (macro): {f1}")


Número total de imágenes en entrenamiento: 10622
Número total de imágenes en validación: 11996

Clases: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


epoch,train_loss,valid_loss,accuracy,recall_score,f1_score,time
0,2.302247,2.302045,0.098700,0.100000,0.017967,00:49
1,2.301726,2.301548,0.112371,0.100000,0.020204,00:38
2,2.302626,2.301409,0.112371,0.100000,0.020204,00:39
3,2.302208,2.301236,0.112371,0.100000,0.020204,00:38
4,2.301320,2.298821,0.112371,0.100000,0.020204,00:38
5,1.713421,1.435649,0.539680,0.530595,0.519328,00:39
6,0.924003,0.819154,0.728576,0.722648,0.716383,00:38
7,0.664440,0.649273,0.789096,0.785656,0.785922,00:39
8,0.570646,0.580640,0.813271,0.810586,0.810700,00:41
9,0.502139,0.503680,0.834945,0.832718,0.832748,00:40



Tiempo de entrenamiento: 1202.26 segundos


Loss: 0.2116
Accuracy: 0.9343
Recall: 0.9336
F1-Score: 0.9336


/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


Loss: 0.18972726166248322
Accuracy: 0.9412059187889099
Recall (macro): 0.9406058278292935
F1 Score (macro): 0.9405060957812188


In [6]:
quantize3("/mnt/homeGPU/haoweihu/code/dataset/distilled_001/mnist/example2", "mnist_dist_quant3_ex2")    

Número total de imágenes en entrenamiento: 10622
Número total de imágenes en validación: 11996

Clases: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


epoch,train_loss,valid_loss,accuracy,recall_score,f1_score,time
0,2.305898,2.304440,0.099116,0.100000,0.018036,00:57
1,2.304226,2.302292,0.112371,0.100000,0.020204,00:39
2,2.302432,2.301279,0.112371,0.100000,0.020204,00:39
3,2.301643,2.301203,0.112371,0.100000,0.020204,00:38
4,2.302357,2.301458,0.112371,0.100000,0.020204,00:39
5,2.301868,2.300449,0.112371,0.100000,0.020204,00:39
6,2.097769,1.884340,0.329110,0.318641,0.226191,00:40
7,1.257071,1.080153,0.638296,0.634514,0.625196,00:39
8,0.813015,0.772941,0.746415,0.743626,0.740208,00:42
9,0.636928,0.634466,0.800517,0.796898,0.795078,00:40



Tiempo de entrenamiento: 1217.10 segundos


Loss: 0.2829
Accuracy: 0.9148
Recall: 0.9139
F1-Score: 0.9140


/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


Loss: 0.2557581961154938
Accuracy: 0.9194711446762085
Recall (macro): 0.9183314161695352
F1 Score (macro): 0.9185119501545378


In [7]:
quantize3("/mnt/homeGPU/haoweihu/code/dataset/distilled_001/mnist/example3", "mnist_dist_quant3_ex3")    

Número total de imágenes en entrenamiento: 10622
Número total de imágenes en validación: 11996

Clases: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


epoch,train_loss,valid_loss,accuracy,recall_score,f1_score,time
0,2.303113,2.303233,0.098616,0.100000,0.017953,00:43
1,2.302667,2.302217,0.112371,0.100000,0.020204,00:43
2,2.302245,2.301256,0.112371,0.100000,0.020204,00:40
3,2.301202,2.301213,0.112371,0.100000,0.020204,00:40
4,2.301782,2.301244,0.112371,0.100000,0.020204,00:39
5,2.302093,2.301302,0.112371,0.100000,0.020204,00:38
6,2.301189,2.299475,0.112371,0.100000,0.020204,00:39
7,1.908019,1.758304,0.369123,0.359103,0.277935,00:44
8,1.438337,1.339400,0.517006,0.506414,0.478994,00:44
9,1.108840,1.044902,0.636712,0.630406,0.626684,00:42



Tiempo de entrenamiento: 1254.99 segundos


Loss: 0.3586
Accuracy: 0.8887
Recall: 0.8874
F1-Score: 0.8874


/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


Loss: 0.34836870431900024
Accuracy: 0.8927283883094788
Recall (macro): 0.8915428178293391
F1 Score (macro): 0.8915154188846159


In [10]:
quantize2("/mnt/homeGPU/haoweihu/code/dataset/distilled_001/mnist/example1", "mnist_dist_quant2_ex1")


Número total de imágenes en entrenamiento: 10622
Número total de imágenes en validación: 11996

Clases: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


epoch,train_loss,valid_loss,accuracy,recall_score,f1_score,time
0,2.303783,2.303691,0.099283,0.100000,0.018063,00:45
1,2.302732,2.302167,0.099283,0.100000,0.018063,00:45
2,2.301783,2.301336,0.112371,0.100000,0.020204,00:45
3,2.302756,2.301247,0.112371,0.100000,0.020204,00:45
4,2.302281,2.301861,0.112371,0.100000,0.020204,00:44
5,2.302225,2.300944,0.112371,0.100000,0.020204,00:45
6,2.196624,1.939861,0.503918,0.492446,0.426937,00:43
7,1.126295,1.028360,0.664055,0.654827,0.637796,00:47
8,0.894610,0.908184,0.706069,0.699721,0.699414,00:43
9,0.757343,0.788696,0.749083,0.745433,0.743844,00:42



Tiempo de entrenamiento: 1289.99 segundos


Loss: 0.3861
Accuracy: 0.8794
Recall: 0.8777
F1-Score: 0.8777


/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


Loss: 0.3567613959312439
Accuracy: 0.8887219429016113
Recall (macro): 0.8869021968846414
F1 Score (macro): 0.886744219729118


In [11]:
quantize2("/mnt/homeGPU/haoweihu/code/dataset/distilled_001/mnist/example2", "mnist_dist_quant2_ex2")

Número total de imágenes en entrenamiento: 10622
Número total de imágenes en validación: 11996

Clases: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


epoch,train_loss,valid_loss,accuracy,recall_score,f1_score,time
0,2.303517,2.302200,0.112371,0.100000,0.020204,00:43
1,2.302279,2.301529,0.112371,0.100000,0.020204,00:41
2,2.301605,2.301270,0.112371,0.100000,0.020204,00:43
3,2.301654,2.301220,0.112371,0.100000,0.020204,00:42
4,2.301502,2.301209,0.112371,0.100000,0.020204,00:42
5,2.303064,2.301397,0.112371,0.100000,0.020204,00:42
6,2.302305,2.301231,0.112371,0.100000,0.020204,00:42
7,2.301597,2.301216,0.112371,0.100000,0.020204,00:43
8,2.301935,2.301191,0.112371,0.100000,0.020204,00:41
9,2.302233,2.301214,0.112371,0.100000,0.020204,00:42


No improvement since epoch 14: early stopping

Tiempo de entrenamiento: 841.96 segundos


Loss: 2.3012
Accuracy: 0.1124
Recall: 0.1000
F1-Score: 0.0202


/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


Loss: 2.3010756969451904
Accuracy: 0.11338140815496445
Recall (macro): 0.1
F1 Score (macro): 0.020367038503058654


In [12]:
quantize2("/mnt/homeGPU/haoweihu/code/dataset/distilled_001/mnist/example3", "mnist_dist_quant2_ex3")

Número total de imágenes en entrenamiento: 10622
Número total de imágenes en validación: 11996

Clases: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


epoch,train_loss,valid_loss,accuracy,recall_score,f1_score,time
0,2.304836,2.303264,0.112371,0.100000,0.020204,00:41
1,2.302625,2.301855,0.112371,0.100000,0.020204,00:41
2,2.301587,2.301328,0.112371,0.100000,0.020204,00:45
3,2.302250,2.301232,0.112371,0.100000,0.020204,00:44
4,2.301877,2.301332,0.112371,0.100000,0.020204,00:43
5,2.302213,2.301526,0.112371,0.100000,0.020204,00:51
6,2.301341,2.301222,0.112371,0.100000,0.020204,01:12
7,2.302769,2.301244,0.112371,0.100000,0.020204,00:59
8,2.302386,2.301244,0.112371,0.100000,0.020204,00:44
9,2.302714,2.301283,0.112371,0.100000,0.020204,00:43


No improvement since epoch 17: early stopping

Tiempo de entrenamiento: 1044.90 segundos


Loss: 2.3012
Accuracy: 0.1124
Recall: 0.1000
F1-Score: 0.0202


/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


Loss: 2.3010101318359375
Accuracy: 0.11368189007043839
Recall (macro): 0.1
F1 Score (macro): 0.020415504991456064


In [ ]:
otsu("/mnt/homeGPU/haoweihu/code/dataset/distilled_001/mnist/example1", "mnist_dist_otsu_ex1")


In [ ]:
otsu("/mnt/homeGPU/haoweihu/code/dataset/distilled_001/mnist/example2", "mnist_dist_otsu_ex2")

In [ ]:
otsu("/mnt/homeGPU/haoweihu/code/dataset/distilled_001/mnist/example3", "mnist_dist_otsu_ex3")

# Distilled 005

In [8]:
quantize3("/mnt/homeGPU/haoweihu/code/dataset/distilled_005/mnist/example1", "mnist_dist5_quant3_ex1")    

Número total de imágenes en entrenamiento: 9814
Número total de imágenes en validación: 11996

Clases: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


epoch,train_loss,valid_loss,accuracy,recall_score,f1_score,time
0,2.303973,2.302978,0.104451,0.100000,0.018915,01:21
1,2.302839,2.301856,0.104451,0.100000,0.018915,00:41
2,2.302077,2.301338,0.112371,0.100000,0.020204,00:41
3,2.302085,2.300916,0.112371,0.100000,0.020204,00:41
4,2.267728,2.161668,0.450567,0.438634,0.343782,00:42
5,1.207892,1.021706,0.644298,0.634924,0.612692,00:40
6,0.877452,0.817724,0.734328,0.729652,0.720624,00:40
7,0.697475,0.661702,0.787512,0.784020,0.779113,00:42
8,0.569114,0.520035,0.839863,0.837916,0.837901,00:40
9,0.455148,0.436533,0.866539,0.865297,0.864453,00:40



Tiempo de entrenamiento: 1332.28 segundos


Loss: 0.1669
Accuracy: 0.9487
Recall: 0.9484
F1-Score: 0.9484


/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


Loss: 0.14633718132972717
Accuracy: 0.9520232081413269
Recall (macro): 0.9515213660522027
F1 Score (macro): 0.9515326335248847


In [9]:
quantize3("/mnt/homeGPU/haoweihu/code/dataset/distilled_005/mnist/example2", "mnist_dist5_quant3_ex2")    

Número total de imágenes en entrenamiento: 9814
Número total de imágenes en validación: 11996

Clases: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


epoch,train_loss,valid_loss,accuracy,recall_score,f1_score,time
0,2.303017,2.303075,0.112371,0.100000,0.020204,01:16
1,2.302288,2.301580,0.112371,0.100000,0.020204,00:39
2,2.301353,2.301235,0.112371,0.100000,0.020204,00:39
3,2.301555,2.301252,0.112371,0.100000,0.020204,00:39
4,2.302190,2.301472,0.112371,0.100000,0.020204,00:39
5,2.303182,2.301356,0.112371,0.100000,0.020204,00:39
6,2.296073,2.280147,0.146465,0.132642,0.061301,00:40
7,1.547655,1.257995,0.582111,0.575016,0.554897,00:39
8,0.946291,0.896502,0.705819,0.704157,0.700682,00:39
9,0.768040,0.737663,0.763921,0.760529,0.755514,00:40



Tiempo de entrenamiento: 1246.72 segundos


Loss: 0.2957
Accuracy: 0.9109
Recall: 0.9101
F1-Score: 0.9101


/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


Loss: 0.27112844586372375
Accuracy: 0.9185696840286255
Recall (macro): 0.9176515275726622
F1 Score (macro): 0.9176918877414918


In [10]:
quantize3("/mnt/homeGPU/haoweihu/code/dataset/distilled_005/mnist/example3", "mnist_dist5_quant3_ex3")    

Número total de imágenes en entrenamiento: 9814
Número total de imágenes en validación: 11996

Clases: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


epoch,train_loss,valid_loss,accuracy,recall_score,f1_score,time
0,2.304336,2.304514,0.112371,0.100000,0.020204,00:43
1,2.302985,2.301795,0.112371,0.100000,0.020204,00:43
2,2.301575,2.301183,0.112371,0.100000,0.020204,00:41
3,2.302244,2.301244,0.112371,0.100000,0.020204,00:42
4,2.302571,2.301170,0.112371,0.100000,0.020204,00:39
5,2.300989,2.298455,0.112371,0.100000,0.020204,00:40
6,1.615489,1.223610,0.603618,0.593681,0.563058,00:39
7,0.946996,0.825054,0.740413,0.734825,0.733210,00:40
8,0.718136,0.653682,0.792764,0.788765,0.787744,00:40
9,0.607448,0.607566,0.821857,0.821587,0.822932,00:40



Tiempo de entrenamiento: 1222.55 segundos


Loss: 0.2176
Accuracy: 0.9346
Recall: 0.9340
F1-Score: 0.9341


/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


Loss: 0.1996885985136032
Accuracy: 0.9372996687889099
Recall (macro): 0.9365622388729473
F1 Score (macro): 0.9366759562354916


In [11]:
quantize2("/mnt/homeGPU/haoweihu/code/dataset/distilled_005/mnist/example1", "mnist_dist5_quant2_ex1")    

Número total de imágenes en entrenamiento: 9814
Número total de imágenes en validación: 11996

Clases: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


epoch,train_loss,valid_loss,accuracy,recall_score,f1_score,time
0,2.304718,2.304994,0.102201,0.100000,0.018545,00:44
1,2.303248,2.302631,0.104451,0.100000,0.018915,00:39
2,2.301662,2.301306,0.112371,0.100000,0.020204,00:40
3,2.301493,2.301309,0.112371,0.100000,0.020204,00:39
4,2.302038,2.301304,0.112371,0.100000,0.020204,00:39
5,2.301960,2.301723,0.112371,0.100000,0.020204,00:39
6,2.302630,2.301539,0.104451,0.100000,0.018915,00:39
7,2.302574,2.301316,0.112371,0.100000,0.020204,00:39
8,2.301148,2.300989,0.112371,0.100000,0.020204,00:39
9,2.277352,2.208062,0.213738,0.202738,0.078145,00:39



Tiempo de entrenamiento: 1217.76 segundos


Loss: 0.4511
Accuracy: 0.8619
Recall: 0.8603
F1-Score: 0.8602


/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


Loss: 0.4224678575992584
Accuracy: 0.8678886294364929
Recall (macro): 0.8660899305999333
F1 Score (macro): 0.8658284339340551


In [12]:
quantize2("/mnt/homeGPU/haoweihu/code/dataset/distilled_005/mnist/example2", "mnist_dist5_quant2_ex2")    

Número total de imágenes en entrenamiento: 9814
Número total de imágenes en validación: 11996

Clases: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


epoch,train_loss,valid_loss,accuracy,recall_score,f1_score,time
0,2.305145,2.304427,0.097533,0.100000,0.017773,00:39
1,2.303896,2.302528,0.097533,0.100000,0.017773,00:39
2,2.301335,2.301511,0.112371,0.100000,0.020204,00:40
3,2.302210,2.301371,0.112371,0.100000,0.020204,00:39
4,2.302380,2.301293,0.112371,0.100000,0.020204,00:40
5,2.302214,2.301098,0.112371,0.100000,0.020204,00:39
6,2.301507,2.300961,0.112371,0.100000,0.020204,00:40
7,2.298544,2.290026,0.112371,0.100000,0.020208,00:40
8,1.859439,1.527296,0.526175,0.517188,0.476518,00:39
9,1.198538,1.079898,0.657553,0.650032,0.636071,00:39



Tiempo de entrenamiento: 1211.74 segundos


Loss: 0.5483
Accuracy: 0.8312
Recall: 0.8288
F1-Score: 0.8286


/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


Loss: 0.5203155875205994
Accuracy: 0.8384414911270142
Recall (macro): 0.8357496891897289
F1 Score (macro): 0.8356148014458809


In [13]:
quantize2("/mnt/homeGPU/haoweihu/code/dataset/distilled_005/mnist/example3", "mnist_dist5_quant2_ex3")    

Número total de imágenes en entrenamiento: 9814
Número total de imágenes en validación: 11996

Clases: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


epoch,train_loss,valid_loss,accuracy,recall_score,f1_score,time
0,2.305041,2.303894,0.099283,0.100000,0.018063,00:40
1,2.303084,2.302199,0.112371,0.100000,0.020204,00:39
2,2.301590,2.301324,0.112371,0.100000,0.020204,00:39
3,2.301957,2.301268,0.112371,0.100000,0.020204,00:39
4,2.302087,2.301263,0.112371,0.100000,0.020204,00:39
5,2.302341,2.301398,0.112371,0.100000,0.020204,00:40
6,2.302135,2.301182,0.112371,0.100000,0.020204,00:40
7,2.300952,2.301230,0.112371,0.100000,0.020204,00:40
8,2.297669,2.288408,0.112371,0.100000,0.020204,00:40
9,1.767435,1.510665,0.507836,0.496287,0.457851,00:41



Tiempo de entrenamiento: 1216.44 segundos


Loss: 0.4643
Accuracy: 0.8530
Recall: 0.8512
F1-Score: 0.8509


/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


Loss: 0.4510238766670227
Accuracy: 0.8581730723381042
Recall (macro): 0.8562487360693011
F1 Score (macro): 0.8558289875245872


In [ ]:
otsu("/mnt/homeGPU/haoweihu/code/dataset/distilled_005/mnist/example1", "mnist_dist5_otsu_ex1")    

In [ ]:
otsu("/mnt/homeGPU/haoweihu/code/dataset/distilled_005/mnist/example2", "mnist_dist5_otsu_ex2")    

In [ ]:
otsu("/mnt/homeGPU/haoweihu/code/dataset/distilled_005/mnist/example3", "mnist_dist5_otsu_ex3")    

# Distilled 001

In [9]:
quantize3("/mnt/homeGPU/haoweihu/code/dataset/distilled_0001/mnist/example1", "mnist_dist01_quant3_ex1")    

Número total de imágenes en entrenamiento: 21714
Número total de imágenes en validación: 11996

Clases: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


epoch,train_loss,valid_loss,accuracy,recall_score,f1_score,time
0,2.302994,2.302217,0.099116,0.100000,0.018036,01:23
1,2.302443,2.302271,0.102201,0.100000,0.018545,01:04
2,2.300454,2.298530,0.132127,0.133897,0.056236,01:04
3,1.074715,1.186942,0.592447,0.582733,0.572444,01:04
4,0.623222,0.853981,0.723741,0.720752,0.716604,01:03
5,0.468700,0.668419,0.781927,0.779283,0.777960,01:04
6,0.358137,0.566842,0.815605,0.812629,0.810709,01:06
7,0.276838,0.441839,0.857619,0.855818,0.855463,01:05
8,0.238532,0.378446,0.876709,0.876210,0.875271,01:04
9,0.170747,0.309056,0.900467,0.898445,0.898284,01:04



Tiempo de entrenamiento: 1961.27 segundos


Loss: 0.1123
Accuracy: 0.9658
Recall: 0.9655
F1-Score: 0.9656


/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


Loss: 0.10027982294559479
Accuracy: 0.969651460647583
Recall (macro): 0.9692360610778803
F1 Score (macro): 0.9693822040741708


In [10]:
quantize3("/mnt/homeGPU/haoweihu/code/dataset/distilled_0001/mnist/example2", "mnist_dist01_quant3_ex2")    

Número total de imágenes en entrenamiento: 21714
Número total de imágenes en validación: 11996

Clases: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


epoch,train_loss,valid_loss,accuracy,recall_score,f1_score,time
0,2.303173,2.304708,0.099283,0.100000,0.018063,01:29
1,2.302516,2.303315,0.102201,0.100000,0.018545,01:04
2,2.301493,2.301965,0.098616,0.100000,0.017953,01:04
3,1.160741,1.063169,0.672974,0.667412,0.663866,01:04
4,0.545697,0.754803,0.754668,0.751833,0.751068,01:03
5,0.438197,0.685743,0.783094,0.782030,0.781639,01:05
6,0.334586,0.535033,0.833444,0.831298,0.831020,01:05
7,0.283422,0.479563,0.848700,0.846484,0.847627,01:07
8,0.238358,0.392917,0.879960,0.878770,0.878769,01:05
9,0.193431,0.337707,0.896299,0.895133,0.895327,01:04



Tiempo de entrenamiento: 1981.77 segundos


Loss: 0.1442
Accuracy: 0.9564
Recall: 0.9560
F1-Score: 0.9560


/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


Loss: 0.1229415088891983
Accuracy: 0.9602363705635071
Recall (macro): 0.9597712049117415
F1 Score (macro): 0.9598000503126503


In [11]:
quantize3("/mnt/homeGPU/haoweihu/code/dataset/distilled_0001/mnist/example3", "mnist_dist01_quant3_ex3")    

Número total de imágenes en entrenamiento: 21714
Número total de imágenes en validación: 11996

Clases: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


epoch,train_loss,valid_loss,accuracy,recall_score,f1_score,time
0,2.302986,2.304801,0.099283,0.100000,0.018063,01:40
1,2.301974,2.303826,0.102201,0.100000,0.018545,01:05
2,2.302383,2.303549,0.099283,0.100000,0.018063,01:05
3,2.302517,2.303347,0.102201,0.100000,0.018545,01:06
4,2.300476,2.299461,0.263921,0.266993,0.176208,01:06
5,1.012365,1.144407,0.631127,0.625770,0.629819,01:04
6,0.631400,0.873250,0.723908,0.721925,0.720974,01:05
7,0.474207,0.716420,0.781010,0.777985,0.777926,01:05
8,0.357731,0.571503,0.837029,0.835784,0.834344,01:04
9,0.308548,0.504601,0.854535,0.853970,0.853964,01:04



Tiempo de entrenamiento: 1978.84 segundos


Loss: 0.2196
Accuracy: 0.9355
Recall: 0.9349
F1-Score: 0.9349


/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


Loss: 0.1923431158065796
Accuracy: 0.9439102411270142
Recall (macro): 0.9433302354312918
F1 Score (macro): 0.943231509431801


In [12]:
quantize2("/mnt/homeGPU/haoweihu/code/dataset/distilled_0001/mnist/example1", "mnist_dist01_quant2_ex1")    

Número total de imágenes en entrenamiento: 21714
Número total de imágenes en validación: 11996

Clases: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


epoch,train_loss,valid_loss,accuracy,recall_score,f1_score,time
0,2.304123,2.302982,0.112371,0.100000,0.020204,00:59
1,2.301641,2.303408,0.097533,0.100000,0.017773,01:03
2,2.302155,2.303174,0.099283,0.100000,0.018063,01:02
3,2.301987,2.303402,0.099283,0.100000,0.018063,01:02
4,2.301285,2.301090,0.174558,0.168683,0.097957,01:01
5,0.966539,0.999974,0.666389,0.659405,0.651814,01:01
6,0.612343,0.855776,0.717489,0.710996,0.704270,01:03
7,0.505379,0.710300,0.772341,0.768748,0.767198,01:00
8,0.381874,0.571976,0.816606,0.814195,0.813682,00:59
9,0.332884,0.494455,0.839613,0.837673,0.836856,01:00



Tiempo de entrenamiento: 1839.82 segundos


Loss: 0.2334
Accuracy: 0.9268
Recall: 0.9261
F1-Score: 0.9260


/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


Loss: 0.21580269932746887
Accuracy: 0.9327924847602844
Recall (macro): 0.9319138838610546
F1 Score (macro): 0.9318428627021076


In [13]:
quantize2("/mnt/homeGPU/haoweihu/code/dataset/distilled_0001/mnist/example2", "mnist_dist01_quant2_ex2")    

Número total de imágenes en entrenamiento: 21714
Número total de imágenes en validación: 11996

Clases: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


epoch,train_loss,valid_loss,accuracy,recall_score,f1_score,time
0,2.303820,2.305275,0.099116,0.100000,0.018036,01:00
1,2.302019,2.303530,0.102201,0.100000,0.018545,00:59
2,2.302338,2.303120,0.102201,0.100000,0.018545,00:59
3,2.302312,2.303679,0.102201,0.100000,0.018545,01:00
4,2.302532,2.302929,0.099116,0.100000,0.018036,01:00
5,2.301739,2.303397,0.099283,0.100000,0.018063,01:01
6,1.852885,1.795559,0.351867,0.341115,0.281622,01:00
7,1.171270,1.314065,0.526259,0.519038,0.500363,01:00
8,0.763181,0.930551,0.695565,0.692512,0.690881,01:01
9,0.552246,0.767060,0.741497,0.738647,0.737322,01:00



Tiempo de entrenamiento: 1829.53 segundos


Loss: 0.2902
Accuracy: 0.9091
Recall: 0.9083
F1-Score: 0.9082


/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


Loss: 0.2589941620826721
Accuracy: 0.9221754670143127
Recall (macro): 0.9213193179452732
F1 Score (macro): 0.9210951562506546


In [14]:
quantize2("/mnt/homeGPU/haoweihu/code/dataset/distilled_0001/mnist/example3", "mnist_dist01_quant2_ex3")    

Número total de imágenes en entrenamiento: 21714
Número total de imágenes en validación: 11996

Clases: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


epoch,train_loss,valid_loss,accuracy,recall_score,f1_score,time
0,2.303403,2.301742,0.112371,0.100000,0.020204,01:02
1,2.302370,2.302373,0.102201,0.100000,0.018545,01:01
2,2.301860,2.302320,0.102201,0.100000,0.018545,01:02
3,2.302154,2.302932,0.102201,0.100000,0.018545,01:02
4,2.284585,2.254765,0.280760,0.275639,0.203439,01:01
5,0.842797,0.957097,0.684228,0.678975,0.676151,00:59
6,0.612045,0.801822,0.742914,0.739362,0.735952,01:00
7,0.499583,0.692611,0.782761,0.780359,0.778920,00:59
8,0.395955,0.550395,0.836862,0.835262,0.835705,01:00
9,0.297031,0.464726,0.864288,0.863402,0.862638,01:00



Tiempo de entrenamiento: 1838.40 segundos


Loss: 0.1836
Accuracy: 0.9451
Recall: 0.9447
F1-Score: 0.9447


/mnt/homeGPU/haoweihu/env/lib/python3.9/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


Loss: 0.15035612881183624
Accuracy: 0.9541265964508057
Recall (macro): 0.9536143067266906
F1 Score (macro): 0.9536230647576351


In [ ]:
otsu("/mnt/homeGPU/haoweihu/code/dataset/distilled_0001/mnist/example1", "mnist_dist01_otsu_ex1")    

Número total de imágenes en entrenamiento: 21714
Número total de imágenes en validación: 11996

Clases: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


epoch,train_loss,valid_loss,accuracy,recall_score,f1_score,time


In [ ]:
otsu("/mnt/homeGPU/haoweihu/code/dataset/distilled_0001/mnist/example2", "mnist_dist01_otsu_ex2")    

In [ ]:
otsu("/mnt/homeGPU/haoweihu/code/dataset/distilled_0001/mnist/example3", "mnist_dist01_otsu_ex3")    